In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
import pickle
import os
import pandas as pd
import random
import PIL as image_lib
import tensorflow as tf
import seaborn as sns
from PIL import Image
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image
import numpy
#load the trained model to classify sign
from keras.models import load_model

In [3]:
# Function to resize an image with aspect ratio preservation
def resize_image_with_aspect_ratio(image, target_size=(64, 64)):
    # Convert the image from BGR to RGB color space
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Get the original image dimensions
    original_height, original_width = image.shape[:2]

    # Calculate the aspect ratio of the original image
    aspect_ratio = original_width / original_height

    # Calculate new dimensions while maintaining aspect ratio
    target_width, target_height, _ = target_size
    if aspect_ratio > 1:
        new_width = target_width
        new_height = int(target_width / aspect_ratio)
    else:
        new_height = target_height
        new_width = int(target_height * aspect_ratio)

    # Resize the image using bilinear interpolation to maintain colors
    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

    # Create an empty canvas of the target size
    canvas = np.zeros((target_height, target_width, 3), dtype=np.uint8)

    # Calculate the position to paste the resized image
    x_offset = (target_width - new_width) // 2
    y_offset = (target_height - new_height) // 2

    # Paste the resized image onto the canvas
    canvas[y_offset:y_offset + new_height, x_offset:x_offset + new_width, :] = resized_image
    
    # Convert the canvas back to BGR color space
    canvas_bgr = cv2.cvtColor(canvas, cv2.COLOR_RGB2BGR)
    
    # Normalize pixel values to the range [0, 1]
    normalized_image = canvas_bgr / 255.0
    
    return normalized_image

In [7]:
model = load_model(r'TrainedModel\resnet50_model_epoch50.h5')

In [4]:
#dictionary to label all traffic signs class.
classes = { 0:'Speed limit (5km/h)', 1:'Speed limit (15km/h)', 2:'Speed limit (30km/h)', 3:'Speed limit (40km/h)', 4:'Speed limit (50km/h)',
           5:'Speed limit (60km/h)', 6:'Speed limit (70km/h)', 7:'Speed limit (80km/h)', 8:'No Straight Through/Left ', 9:'No Straight Through/Right',
           10:'No Straight Through', 11:'No Left Turn', 12:'No Left/Right Turn', 13:'No right Turn', 14:'No Overtaking', 15:'No U-Turn',
           16:'No Vehicles', 17:'No-Honking', 18:'Speed limit (40km/h)', 19:'Speed limit (50km/h)', 20:'Dangerous curve to the right',
           21:'One Way ', 22:'Turn Left ahead', 23:'Turn Left/Right', 24:'Turn right ahead', 25:'Keep Left', 26:'Keep Right', 27:'Roundabout', 
           28:'Mandatory lane for cars', 29:'Honking Mandatory', 30:'Cyclist path', 31:'U-Turn', 32:'Obstacle Warning', 33:'Traffic Light Ahead',
           34:'Cars Not Allowed', 35:'Pedestrain Crossing', 36:'Cyclist Warning', 37:'Child Warning', 38:'Road Right Bend', 39:'Road Left Bend', 
           40:'Steep Descent Ahead', 41:'Steep Ascent  Ahead', 42:'Causion High Voltage', 43:'Uncontrolled Crossroad from Right Warning',
           44:'Uncontrolled Crossroad from Left Warning', 45:'Housing Area', 46:'Double Curve Ahead Warning', 47:'Rail Crossing', 48:'Road Works Ahead',
           49:'Dangerous Road Bends', 50:'Railway Crossing Ahead', 51:'Rear End Collision', 52:'Stop Sign', 53:'No Vehicles Entry',
           54:'No parking', 55:'No Entry', 56:'Yield', 57:'Do not Enter'
          }#dictionary to label all traffic signs class.

In [ ]:
imageDim = (64, 64, 3)
images=[]
top=tk.Tk()
top.geometry('900x700')
top.title('Traffic Sign Classification')
top.configure(background='#516091')
label=Label(top,background='#516091', font=('arial',20,'bold'))
sign_image = Label(top)
def classify(file_path):
    if file_path:
        # Load and preprocess the selected image
        image = cv2.imread(file_path)
        image = resize_image_with_aspect_ratio(image, target_size=imageDim)
        image = np.expand_dims(image, axis=0)  # Add batch dimension
        image = image.astype(np.float32)  # Ensure data type is float32
        
        # Perform classification using the loaded model
        result = model.predict(image)
        # Assuming the model outputs class probabilities, you can find the predicted class
        predicted_class = np.argmax(result)
        print(classes[predicted_class])
        label.configure(foreground='#facbd3', text=classes[predicted_class],)
def show_classify_button(file_path):
    classify_b=Button(top,text="Classify Image",command=lambda: classify(file_path),padx=10,pady=5)
    classify_b.configure(background='#50717b', foreground='white',font=('arial',10,'bold'))
    classify_b.place(relx=0.79,rely=0.46)

def upload_image():
    try:
       file_path=filedialog.askopenfilename()
       uploaded=Image.open(file_path)
       uploaded.thumbnail(((top.winfo_width()/2.25),(top.winfo_height()/2.25)))
       im=ImageTk.PhotoImage(uploaded)
       sign_image.configure(image=im)
       sign_image.image=im
       label.configure(text='')
       show_classify_button(file_path)
    except:
       pass
upload=Button(top,text="Upload Image",command=upload_image,padx=10,pady=5)
upload.configure(background='#50717b', foreground='white',font=('arial',10,'bold'))
upload.pack(side=BOTTOM,pady=50)
sign_image.pack(side=BOTTOM,expand=True)
label.pack(side=BOTTOM,expand=True)
heading = Label(top, text="Check Traffic Sign",pady=20, font=('arial',30,'bold'))
heading.configure(background='#516091',foreground='#74bec1')
heading.pack()
top.mainloop()

1/1 [==============================] - 2s 2s/step
Speed limit (5km/h)
